https://python.langchain.com/docs/use_cases/sql/quickstart/

### **Test the sqldb**

In [1]:
from langchain_community.utilities import SQLDatabase
from pyprojroot import here
import warnings
warnings.filterwarnings("ignore")

**Connecting to the sqldb**

In [2]:
db_path = str(here("data")) + "/sqldb.db"
db = SQLDatabase.from_uri(f"sqlite:///{db_path}")

In [23]:
db

In [ ]:
# validate the connection to the vectordb
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM Artist LIMIT 10;")

sqlite
['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


'[(1, \'AC/DC\'), (2, \'Accept\'), (3, \'Aerosmith\'), (4, \'Alanis Morissette\'), (5, \'Alice In Chains\'), (6, \'Antônio Carlos Jobim\'), (7, \'Apocalyptica\'), (8, \'Audioslave\'), (9, \'BackBeat\'), (10, \'Billy Cobham\'), (11, \'Black Label Society\'), (12, \'Black Sabbath\'), (13, \'Body Count\'), (14, \'Bruce Dickinson\'), (15, \'Buddy Guy\'), (16, \'Caetano Veloso\'), (17, \'Chico Buarque\'), (18, \'Chico Science & Nação Zumbi\'), (19, \'Cidade Negra\'), (20, \'Cláudio Zoli\'), (21, \'Various Artists\'), (22, \'Led Zeppelin\'), (23, \'Frank Zappa & Captain Beefheart\'), (24, \'Marcos Valle\'), (25, \'Milton Nascimento & Bebeto\'), (26, \'Azymuth\'), (27, \'Gilberto Gil\'), (28, \'João Gilberto\'), (29, \'Bebel Gilberto\'), (30, \'Jorge Vercilo\'), (31, \'Baby Consuelo\'), (32, \'Ney Matogrosso\'), (33, \'Luiz Melodia\'), (34, \'Nando Reis\'), (35, \'Pedro Luís & A Parede\'), (36, \'O Rappa\'), (37, \'Ed Motta\'), (38, \'Banda Black Rio\'), (39, \'Fernanda Porto\'), (40, \'Os Ca

### **Test the access to the environment variables**

In [2]:
from dotenv import load_dotenv
import os
print("Environment variables are loaded:", load_dotenv("C:\\Users\\sourabh.mohite\\LLM_Repo\\Advanced-QA-and-RAG-Series_2025\\Q&A-and-RAG-with-SQL-and-TabularData\\.env"))
print("test by reading a variable:", os.getenv("OPENAI_API_TYPE"))

Environment variables are loaded: True
test by reading a variable: azure


### **Test your GPT model**

In [5]:
from openai import AzureOpenAI

messages = [
    {"role": "system", "content": str(
        "You are a helpful assistant"
    )},
    {"role": "user", "content": str("hello")}
]

client = AzureOpenAI(
    api_version=os.getenv("OPENAI_API_VERSION"),
    azure_endpoint=os.getenv("OPENAI_API_BASE"),
    api_key=os.getenv("OPENAI_API_KEY"),
    azure_deployment=os.getenv("GET_DEPLOYMENT_NAME")
)

response = client.chat.completions.create(
    model=os.getenv("gpt_deployment_name"),
    messages=messages
)

print(response.choices[0].message.content)



Hello! How can I assist you today?


### **1. SQL query chain**

In [3]:
from langchain_openai import AzureChatOpenAI

openai_api_version = os.getenv("OPENAI_API_VERSION")
deployment_name = os.getenv("GET_DEPLOYMENT_NAME")
model_name = os.getenv("MODEL_NAME")
azure_openai_endpoint = os.getenv("OPENAI_API_BASE")
azure_openai_api_key = os.getenv("OPENAI_API_KEY")

llm = AzureChatOpenAI(
    openai_api_version=openai_api_version,
    azure_deployment=deployment_name,
    model_name=model_name,
    azure_endpoint=azure_openai_endpoint,
    api_key=azure_openai_api_key,
    temperature=0.0)

In [6]:
from langchain.chains import create_sql_query_chain

chain = create_sql_query_chain(llm, db)
response = chain.invoke({"question": "How many employees are there"})
print(response)

SELECT COUNT(*) AS "Total Employees" FROM employees


Execute the query to make sure it’s valid

In [29]:
db.run(response)

'[(8,)]'

In [9]:
chain.get_prompts()[0].pretty_print()

You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result

### **Add QuerySQLDataBaseTool to the chain**
Execute SQL query

**This is the most dangerous part of creating a SQL chain.** Consider carefully if it is OK to run automated queries over your data. Minimize the database connection permissions as much as possible. Consider adding a human approval step to you chains before query execution (see below).

We can use the QuerySQLDatabaseTool to easily add query execution to our chain:

In [4]:
db = SQLDatabase.from_uri(f"sqlite:///C:/Users/sourabh.mohite/LLM_Repo/Advanced-QA-and-RAG-Series_2025/Q&A-and-RAG-with-SQL-and-TabularData/data/uploaded_files_sqldb.db")


In [28]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

write_query = create_sql_query_chain(llm, db)
execute_query = QuerySQLDataBaseTool(db=db)

chain = write_query | execute_query

chain.invoke({"question": "pivot on claim_by_model with column heading"})

"[('2024/01', 1, 1, 1), ('2024/02', None, None, None), ('2024/03', 1, None, 2), ('2024/04', 1, None, 2), ('2024/05', 1, 1, 1), ('2024/06', None, None, 4), ('2024/07', 1, 1, 2), ('2024/08', 4, 2, 3), ('2024/09', 3, 2, 1), ('2024/10', 3, None, 7), ('2024/11', None, None, 2)]"

### **Answer the question in a user friendly manner**

In [20]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

answer = answer_prompt | llm | StrOutputParser()
chain = (
    RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query") | execute_query
    )
    | answer
)

# chain.invoke({"question": "How many employees are there"})
chain.invoke({"question": "display claim models data"})

BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 16384 tokens. However, your messages resulted in 62801 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}

### **2. Agents**

Agent which provides a more flexible way of interacting with SQL databases. The main advantages of using the SQL Agent are:

- It can answer questions based on the databases’ schema as well as on the databases’ content (like describing a specific table).
- It can recover from errors by running a generated query, catching the traceback and regenerating it correctly.
- It can answer questions that require multiple dependent queries.
- It will save tokens by only considering the schema from relevant tables.

To initialize the agent, we use create_sql_agent function. This agent contains the SQLDatabaseToolkit which contains tools to:

- Create and execute queries
- Check query syntax
- Retrieve table descriptions
- …

In [15]:
from langchain_community.agent_toolkits import create_sql_agent

agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)

In [21]:
agent_executor.invoke(
    {
        "input": "display all data from claim_by_model"
    }
)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


Claim_By_Model, titanic
Invoking: `sql_db_schema` with `{'table_names': 'Claim_By_Model'}`



CREATE TABLE "Claim_By_Model" (
	"claim month" TEXT, 
	"claimed model" TEXT, 
	"claim count" BIGINT
)

/*
3 rows from Claim_By_Model table:
claim month	claimed model	claim count
2024/01	2-SC-51A	1
2024/01	2-SC-52A	1
2024/01	2-SC-53A	1
*/
Invoking: `sql_db_query` with `{'query': 'SELECT * FROM Claim_By_Model'}`


[('2024/01', '2-SC-51A', 1), ('2024/01', '2-SC-52A', 1), ('2024/01', '2-SC-53A', 1), ('2024/01', '2-SC-53B', 1), ('2024/01', '2-SO-51A', 1), ('2024/01', '2-SO-51B', 1), ('2024/01', '2-SO-52A', 1), ('2024/01', '2-SO-52B', 2), ('2024/01', '2-iPhone11-64', 1), ('2024/01', '2-iPhone12-128', 1), ('2024/01', '2-iPhone12PM-1', 1), ('2024/01', '2-iPhone12m-12', 1), ('2024/01', '2-iPhoneSE2-64', 1), ('2024/01', '2-iPhoneX-64', 1), ('2024/01', '2-iPhoneXS-64', 1), ('2024/01', '2211iPadPr-128s', 8), ('2024/01

BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 16384 tokens. However, your messages resulted in 101124 tokens (100808 in the messages, 316 in the functions). Please reduce the length of the messages or functions.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}

In [23]:
agent_executor.invoke(
    {
        "input": "List the total sales per country. Which country's customers spent the most?"
    }
)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
Invoking: `sql_db_schema` with `{'table_names': 'Invoice,Customer'}`



CREATE TABLE "Customer" (
	"CustomerId" INTEGER NOT NULL, 
	"FirstName" NVARCHAR(40) NOT NULL, 
	"LastName" NVARCHAR(20) NOT NULL, 
	"Company" NVARCHAR(80), 
	"Address" NVARCHAR(70), 
	"City" NVARCHAR(40), 
	"State" NVARCHAR(40), 
	"Country" NVARCHAR(40), 
	"PostalCode" NVARCHAR(10), 
	"Phone" NVARCHAR(24), 
	"Fax" NVARCHAR(24), 
	"Email" NVARCHAR(60) NOT NULL, 
	"SupportRepId" INTEGER, 
	PRIMARY KEY ("CustomerId"), 
	FOREIGN KEY("SupportRepId") REFERENCES "Employee" ("EmployeeId")
)

/*
3 rows from Customer table:
CustomerId	FirstName	LastName	Company	Address	City	State	Country	PostalCode	Phone	Fax	Email	SupportRepId
1	Luís	Gonçalves	Embraer - Empresa Brasileira de Aeronáutica S.A.	Av. Brigadeiro Faria Lima, 2170	São José 

{'input': "List the total sales per country. Which country's customers spent the most?",
 'output': 'The total sales per country are as follows:\n\n1. USA: $523.06\n2. Canada: $303.96\n3. France: $195.10\n4. Brazil: $190.10\n5. Germany: $156.48\n6. United Kingdom: $112.86\n7. Czech Republic: $90.24\n8. Portugal: $77.24\n9. India: $75.26\n10. Chile: $46.62\n\nTo find the country whose customers spent the most, we can see that the USA has the highest total sales of $523.06. Therefore, the country whose customers spent the most is the USA.'}

In [12]:
agent_executor.invoke({"input": "Describe the playlisttrack table"})
# agent_executor.invoke("Describe the playlisttrack table")

NameError: name 'agent_executor' is not defined